In [21]:
spark

res12: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@781ab791


In [22]:
spark.version

res13: String = 2.3.1


In [23]:
val sqlContext = new org.apache.spark.sql.SQLContext(sc)
import sqlContext.implicits._

sqlContext: org.apache.spark.sql.SQLContext = org.apache.spark.sql.SQLContext@73fa8021
import sqlContext.implicits._


# DoE Dataset Load concise

In [24]:
val inHeader = sc.textFile("MERGED2016_17_PP.csv").map(x => x + ",YEAR").map(line =>line.split(","))

val in2016 = sc.textFile("MERGED2016_17_PP.csv").map(x => x + ",2016").map(line =>line.split(","))

val in2015 = sc.textFile("MERGED2015_16_PP.csv").map(x => x + ",2015").map(line =>line.split(","))

val in2014 = sc.textFile("MERGED2014_15_PP.csv").map(x => x + ",2014").map(line =>line.split(","))

val in2013 = sc.textFile("MERGED2013_14_PP.csv").map(x => x + ",2013").map(line =>line.split(","))

val in2012 = sc.textFile("MERGED2012_13_PP.csv").map(x => x + ",2012").map(line =>line.split(","))

val in2011 = sc.textFile("MERGED2011_12_PP.csv").map(x => x + ",2011").map(line =>line.split(","))

val in2010 = sc.textFile("MERGED2010_11_PP.csv").map(x => x + ",2010").map(line =>line.split(","))

val in2009 = sc.textFile("MERGED2009_10_PP.csv").map(x => x + ",2009").map(line =>line.split(","))

val in2008 = sc.textFile("MERGED2008_09_PP.csv").map(x => x + ",2008").map(line =>line.split(","))

val in2007 = sc.textFile("MERGED2007_08_PP.csv").map(x => x + ",2007").map(line =>line.split(","))


inHeader: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[69] at map at <console>:35
in2016: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[73] at map at <console>:37
in2015: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[77] at map at <console>:39
in2014: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[81] at map at <console>:41
in2013: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[85] at map at <console>:43
in2012: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[89] at map at <console>:45
in2011: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[93] at map at <console>:47
in2010: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[97] at map at <console>:49
in2009: org.apache.spark.rdd.RDD[Array[Stri...

## Compile the 10 year RDD.

In [25]:
val scoreCard10Year = sc.union(in2016, in2015, in2014, in2013, in2012, in2011, in2010, in2009, in2008, in2007)

scoreCard10Year: org.apache.spark.rdd.RDD[Array[String]] = UnionRDD[110] at union at <console>:46


### Make sure that my added year column is present in all of the load RDDs. 

In [26]:
// scoreCard10Year.map(x => x(1899)).map(_.mkString("")).take(1) // I verified index 1899 in all of my load RDDs

In [27]:
//scoreCard10Year.coalesce(1).map(_.mkString(",")).saveAsTextFile("scoreCard10Year") 
// I exported a csv of the compiled dataset to play around with on the databricks platform. I decided to do that after watching the Spark videos in the week 12 assignment. 


# Create a header index of the dataset

In [28]:
val header2016 = sc.parallelize(inHeader.take(1))

val header2016Index = header2016.map(_.mkString(",")).flatMap(line => line.split(",")).zipWithIndex

// header2016Index.coalesce(1).saveAsTextFile("index") //Textfiles only made once


header2016: org.apache.spark.rdd.RDD[Array[String]] = ParallelCollectionRDD[111] at parallelize at <console>:41
header2016Index: org.apache.spark.rdd.RDD[(String, Long)] = ZippedWithIndexRDD[114] at zipWithIndex at <console>:43


## Count of rows for the compiled DoE Scorecard

In [29]:
scoreCard10Year.count()

res16: Long = 74181


### Number of Schools in the DoE dataset

In [30]:
scoreCard10Year.map(x=> x(1)).distinct().count()

res17: Long = 9743


---

# St Thomas RDD  
This RDD contains all of the St. Thomas results over the years in RDD and dataframe format.

Bummed I have to switch back to SQL for this stuff....

In [31]:
val ust = scoreCard10Year.filter(x => x(0) contains "174914")
// df.filter(df("age") > 21).show()

ust: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[119] at filter at <console>:48


In [32]:
ust.take(1)

res18: Array[Array[String]] = Array(Array(174914, 00234500, 002345, University of St Thomas, Saint Paul, MN, 55105-1078, Higher Learning Commission, www.stthomas.edu, www.stthomas.edu/financialaid/undergraduate/resources/netpricecalculator/, NULL, 0, 1, 1, 3, 4, 2, 27, 4, 11, NULL, 44.943439, -93.188986, 17, 10, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 30, 0.8266, 0.82655521620318, 500, 660, 550, 630, NULL, NULL, 580, 590, NULL, 24, 29, 23, 29, 24, 28, NULL, NULL, 27, 26, 26, NULL, 1204, 1204, 0, 0.0037, 0, 0.0007, 0.0504, 0, 0.0212, 0, 0.0358, 0.0651, 0, 0.0073, 0.0066, 0, 0.0139, 0.0066, 0, 0.0658, 0.0117, 0, 0.0102, 0, 0.0417, 0, 0.0197, 0, 0.0336, 0, 0.0117, 0.0621, 0, 0, 0, 0, 0.0058, 0.0417, 0.4751, 0.0095, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...

In [33]:
ust.count()

res19: Long = 10


In [34]:
val ustHeader = header2016.union(ust)
// ustHeader.map(_.mkString(",")).coalesce(1).saveAsTextFile("ustHeader")

ustHeader: org.apache.spark.rdd.RDD[Array[String]] = UnionRDD[120] at union at <console>:53


---

# Regional Peers RDD

This RDD filters region 4 (upper midwest/plains), Schools that offer graduate degrees as the highest degree, and a carnegie classification of 4 year, medium size, highly residential. 

In [35]:
val regionalPeers = scoreCard10Year.filter(x => x(18) == "4").filter(x => x(15) == "4").filter(x => x(25) == "13")

regionalPeers: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[123] at filter at <console>:48


In [36]:
regionalPeers.count()

res20: Long = 20


Now we see who we're dealing with. 

In [37]:
regionalPeers.filter(x => x(1899) == "2016").map(x => x(3)).collect()


res21: Array[String] = Array(Emporia State University, Bemidji State University, Hamline University, Minnesota State University Moorhead, Saint Mary's University of Minnesota, University of St Thomas, St Catherine University, Southwest Minnesota State University, Winona State University, Drury University, Missouri Western State University, Missouri University of Science and Technology, Truman State University, Northwest Missouri State University, Southeast Missouri State University, Webster University, University of Nebraska at Kearney, Wayne State College, Black Hills State University, University of South Dakota)


---

# National Peers RDD

In [46]:
val nationalPeers = scoreCard10Year.filter(x => x(15) == "4").filter(x => x(25) == "13")

nationalPeers: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[154] at filter at <console>:48


---

# Minnesota Schools  
This rdd contains all of the Minnesota universities for a statewide comparison

In [40]:
val mnColleges = scoreCard10Year.filter( x => x(5) contains "MN").filter( x => x(1899) contains "2016")

mnColleges: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[130] at filter at <console>:48


In [41]:
val mnColLoc = mnColleges.filter(x => x(1899) contains "2016").filter(x => x(36) != "NULL").map(x => x(3) + "," + x(21) + "," + x(22) + "," + x(36))

mnColLoc: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[133] at map at <console>:50


In [42]:
// mnColLoc.take(5).foreach(println) // Only needs to run once.

---

# SAT Scores

### St Thomas

In [83]:
ust.filter(x => x(1899) == "2016").map(x => x(59)).filter(x => x != "NULL").map(_.toInt).mean()

res63: Double = 1204.0


### Minnesota SAT average

In [43]:
mnColleges.filter(x => x(1899) == "2016").map(x => x(59)).filter(x => x != "NULL").map(_.toInt).mean()

res25: Double = 1102.0666666666668


### Regional Peers SAT Average

In [44]:
regionalPeers.filter(x => x(1899) == "2016").map(x => x(59)).filter(x => x != "NULL").map(_.toInt).mean()


res26: Double = 1080.7222222222224


### National Peers SAT Average

In [47]:
nationalPeers.filter(x => x(1899) == "2016").map(x => x(59)).filter(x => x != "NULL").filter(x => x != "SAT_AVG").map(_.toInt).mean()

res28: Double = 1040.932432432432


### National SAT Average

In [45]:
scoreCard10Year.filter(x => x(1899) == "2016").map(x => x(59)).filter(x => x != "NULL").filter(x => x != "SAT_AVG").map(_.toInt).mean()

res27: Double = 1059.2575757575776


### Catholic Schools Average

In [48]:
scoreCard10Year.filter(x => x(1899) == "2016").filter(x => x(35) == "30").map(x => x(59)).filter(x => x != "NULL").filter(x => x != "SAT_AVG").map(_.toInt).mean()

res29: Double = 1057.0273972602745


---

# % Pell Grants

### St. Thomas

In [82]:
ust.filter(x => x(1899) == "2016").map(x => x(385)).filter(x => x != "NULL").map(_.toFloat).mean()

res62: Double = 0.17159999907016754


### Minnesota average

In [49]:
mnColleges.filter(x => x(1899) == "2016").map(x => x(385)).filter(x => x != "NULL").map(_.toFloat).mean()

res30: Double = 0.41704077696771297


### Regional Peers Average

In [50]:
regionalPeers.filter(x => x(1899) == "2016").map(x => x(385)).filter(x => x != "NULL").map(_.toFloat).mean()


res31: Double = 0.30709999911487107


### National Peers Average

In [53]:
nationalPeers.filter(x => x(1899) == "2016").map(x => x(385)).filter(x => x != "NULL").filter(x => x != "PCTPELL").map(_.toFloat).mean()

res34: Double = 0.36608650261090575


### National Average

In [55]:
scoreCard10Year.filter(x => x(1899) == "2016").map(x => x(385)).filter(x => x != "NULL").filter(x => x != "PCTPELL").map(_.toFloat).mean()

res35: Double = 0.49741185117230924


### Catholic Schools Average

In [57]:
scoreCard10Year.filter(x => x(1899) == "2016").filter(x => x(35) == "30").map(x => x(385)).filter(x => x != "NULL").filter(x => x != "PCTPELL").map(_.toFloat).mean()

res37: Double = 0.3206765762454752


---

# % Federal Loans

### St Thomas

In [81]:
ust.filter(x => x(1899) == "2016").map(x => x(437)).filter(x => x != "NULL").map(_.toFloat).mean()

res61: Double = 0.5277000069618225


### Minnesota average

In [58]:
mnColleges.filter(x => x(1899) == "2016").map(x => x(437)).filter(x => x != "NULL").map(_.toFloat).mean()

res38: Double = 0.534766020126713


### Regional Peers Average

In [59]:
regionalPeers.filter(x => x(1899) == "2016").map(x => x(437)).filter(x => x != "NULL").map(_.toFloat).mean()


res39: Double = 0.5363800004124641


### National Peers Average

In [62]:
nationalPeers.filter(x => x(1899) == "2016").map(x => x(437)).filter(x => x != "NULL").filter(x => x != "PCTFLOAN").map(_.toFloat).mean()

res42: Double = 0.5871932504732915


### National Average

In [63]:
scoreCard10Year.filter(x => x(1899) == "2016").map(x => x(437)).filter(x => x != "NULL").filter(x => x != "PCTFLOAN").map(_.toFloat).mean()

res43: Double = 0.4905751478348196


### Catholic Schools Average

In [64]:
scoreCard10Year.filter(x => x(1899) == "2016").filter(x => x(35) == "30").map(x => x(437)).filter(x => x != "NULL").filter(x => x != "PCTFLOAN").map(_.toFloat).mean()

res44: Double = 0.5970229740581802


---

# Median Debt

### St. Thomas

In [87]:
ust.filter(x => x(1899) == "2016").map(x => x(1504)).filter(x => x != "NULL").filter(x => x != "PrivacySuppressed").map(_.toFloat).mean()

res66: Double = 25000.0


### Minnesota average

In [74]:
mnColleges.filter(x => x(1899) == "2016").map(x => x(1504)).filter(x => x != "NULL").filter(x => x != "PrivacySuppressed").map(_.toFloat).mean()

res54: Double = 18007.62719298246


### Regional Peers Average

In [75]:
regionalPeers.filter(x => x(1899) == "2016").map(x => x(1504)).filter(x => x != "NULL").map(_.toFloat).mean()


res55: Double = 22844.75


### National Peers Average

In [76]:
nationalPeers.filter(x => x(1899) == "2016").map(x => x(1504)).filter(x => x != "NULL").filter(x => x != "PCTPELL").filter(x => x != "PrivacySuppressed").map(_.toFloat).mean()

res56: Double = 23838.73148148149


### National Average

In [77]:
scoreCard10Year.filter(x => x(1899) == "2016").map(x => x(1504)).filter(x => x != "NULL").filter(x => x != "GRAD_DEBT_MDN").filter(x => x != "PrivacySuppressed").map(_.toFloat).mean()

res57: Double = 16264.447147950095


### Catholic Schools Average

In [78]:
scoreCard10Year.filter(x => x(1899) == "2016").filter(x => x(35) == "30").map(x => x(1504)).filter(x => x != "NULL").filter(x => x != "GRAD_DEBT_MDN").filter(x => x != "PrivacySuppressed").map(_.toFloat).mean()

res58: Double = 24354.482673267335


---

### Create and export RDD to CSV for mapping 

In [84]:
val mnColLocHead = Array("Name,Lat,Long,AdmRate")
val mnLocRDD = sc.parallelize(mnColLocHead)
val mnColLocFin = mnLocRDD.union(mnColLoc)
//mnColLocFin.coalesce(1).saveAsTextFile("MnSchools") // only run once

mnColLocHead: Array[String] = Array(Name,Lat,Long,AdmRate)
mnLocRDD: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[397] at parallelize at <console>:53
mnColLocFin: org.apache.spark.rdd.RDD[String] = UnionRDD[398] at union at <console>:54
